In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Politician Trades/all_trades_add_days.csv')
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df

,Politician Name,Party,Chamber,State,Issuer Name,Ticker,Publication Date,Transaction Date,Reporting Gap,Owner,...,Price_-90_days,Price_-60_days,Price_-40_days,Price_-20_days,Price_-10_days,Price_10_days,Price_20_days,Price_40_days,Price_60_days,Price_90_days
0,Tom Carper,Democrat,Senate,DE,Valero Energy Corp,VLO:US,2024-05-01,2024-04-11,19,Spouse,...,131.500000,142.529999,143.279999,169.639999,172.639999,165.899994,155.660004,NaN,NaN,NaN
1,Tom Carper,Democrat,Senate,DE,Enbridge Inc,ENB:US,2024-05-01,2024-04-11,19,Spouse,...,36.840000,34.889999,34.799999,35.450001,35.830002,35.349998,35.590000,NaN,NaN,NaN
2,Tom Carper,Democrat,Senate,DE,JPMorgan Chase & Co,JPM:US,2024-05-01,2024-04-01,29,Spouse,...,172.080002,173.729996,180.899994,189.839996,196.619995,195.429993,189.410004,NaN,NaN,NaN
3,Dan Newhouse,Republican,House,WA,Accenture PLC,ACN:US,2024-04-26,2024-04-10,15,Spouse,...,347.970001,371.670013,380.989990,345.029999,339.170013,316.880005,300.910004,NaN,NaN,NaN
4,Dan Newhouse,Republican,House,WA,Analog Devices Inc,ADI:US,2024-04-26,2024-04-10,15,Spouse,...,188.960007,195.020004,196.160004,195.149994,196.619995,183.360001,200.610001,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35036,Mark Green,Republican,House,TN,NGL Energy Partners LP,NGL:US,2021-03-21,2021-03-17,3,Undisclosed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35037,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP:US,2021-03-20,2021-03-19,0,Spouse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35038,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP:US,2021-03-20,2021-03-19,0,Undisclosed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35039,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP:US,2021-03-20,2021-03-19,0,Spouse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35041 entries, 0 to 35040
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Politician Name            35041 non-null  object        
 1   Party                      35041 non-null  object        
 2   Chamber                    35041 non-null  object        
 3   State                      35041 non-null  object        
 4   Issuer Name                35041 non-null  object        
 5   Ticker                     35041 non-null  object        
 6   Publication Date           35041 non-null  object        
 7   Transaction Date           35041 non-null  datetime64[ns]
 8   Reporting Gap              35041 non-null  int64         
 9   Owner                      35041 non-null  object        
 10  Transaction Type           35041 non-null  object        
 11  Value Range                35041 non-null  object        
 12  Pric

In [ ]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm

file_path = '/content/drive/MyDrive/Politician Trades/all_trades_add_days.csv'

def fetch_closest_price(ticker_symbol, date):
    start_date = date - pd.Timedelta(days=3)
    end_date = date + pd.Timedelta(days=3)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    try:
        data = yf.download(ticker_symbol, start=start_date_str, end=end_date_str, progress=False)
        data.dropna(subset=['Close'], inplace=True)
        data['date_diff'] = abs((data.index - date).days)
        data.sort_values('date_diff', inplace=True)
        return data.iloc[0]['Close'] if not data.empty else None
    except Exception as e:
        print(f"Error downloading data for {ticker_symbol} on {date.date()}: {e}")
        return None

days_offsets = [-90, -60, -40, -20, -10, 0, 10, 20, 40, 60, 90]

def process_data(start_index=28300):
    for index, row in tqdm(df.iloc[start_index:].iterrows(), total=(df.shape[0] - start_index), desc='Processing rows'):
        if (index - start_index) % 100 == 0:
            df.to_csv(file_path, index=False)  # Save every 100 rows after start_index

        ticker_symbol = row['US Ticker']
        transaction_date = pd.to_datetime(row['Transaction Date'])

        for offset in days_offsets:
            col_name = f'Price_{offset}_days'
            if pd.isna(row[col_name]):  # Check if data already exists
                offset_date = transaction_date + pd.Timedelta(days=offset)
                closest_price = fetch_closest_price(ticker_symbol, offset_date)
                if closest_price is not None:
                    df.at[index, col_name] = closest_price
                else:
                    print(f"No valid data found for {ticker_symbol} {offset_date.date()}")

    df.to_csv(file_path, index=False)  # Save the DataFrame after the loop

# Assuming 'df' is your DataFrame loaded beforehand
process_data()
print("Price updates complete.")


Processing rows:   0%|          | 19/6741 [00:17<1:40:52,  1.11it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BROS']: Exception("%ticker%: Data doesn't exist for startDate = 1626235200, endDate = 1626753600")
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BROS']: Exception("%ticker%: Data doesn't exist for startDate = 1628827200, endDate = 1629345600")


Error downloading data for BROS on 2021-07-17: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
No valid data found for BROS 2021-07-17
Error downloading data for BROS on 2021-08-16: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
No valid data found for BROS 2021-08-16


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BROS']: Exception("%ticker%: Data doesn't exist for startDate = 1630555200, endDate = 1631073600")


Error downloading data for BROS on 2021-09-05: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
No valid data found for BROS 2021-09-05


Processing rows:   0%|          | 22/6741 [00:25<3:22:34,  1.81s/it]ERROR:yfinance:Failed to get ticker 'BRK/B' reason: Expecting value: line 1 column 1 (char 0)
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK/B']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:Failed to get ticker 'BRK/B' reason: Expecting value: line 1 column 1 (char 0)
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK/B']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:Failed to get ticker 'BRK/B' reason: Expecting value: line 1 column 1 (char 0)
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK/B']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:Failed to get ticker 'BRK/B' reason: Expecting value: line 1 column 1 (char 0)
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK/B']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:Failed to get ticker 'BRK/B' reason: 

...

Error downloading data for nan on 2020-12-16: 'float' object has no attribute 'replace'
No valid data found for nan 2020-12-16
Error downloading data for nan on 2021-01-15: 'float' object has no attribute 'replace'
No valid data found for nan 2021-01-15
Error downloading data for nan on 2021-02-04: 'float' object has no attribute 'replace'
No valid data found for nan 2021-02-04
Error downloading data for nan on 2021-02-24: 'float' object has no attribute 'replace'
No valid data found for nan 2021-02-24
Error downloading data for nan on 2021-03-06: 'float' object has no attribute 'replace'
No valid data found for nan 2021-03-06
Error downloading data for nan on 2021-03-16: 'float' object has no attribute 'replace'
No valid data found for nan 2021-03-16
Error downloading data for nan on 2021-03-26: 'float' object has no attribute 'replace'
No valid data found for nan 2021-03-26
Error downloading data for nan on 2021-04-05: 'float' object has no attribute 'replace'
No valid data found for

Processing rows: 100%|█████████▉| 6740/6741 [1:32:21<00:00,  2.11it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MMP']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MMP']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MMP']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MMP']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MMP']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MMP']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MMP']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MMP']: Exc

Error downloading data for MMP on 2020-12-16: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
No valid data found for MMP 2020-12-16
Error downloading data for MMP on 2021-01-15: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
No valid data found for MMP 2021-01-15
Error downloading data for MMP on 2021-02-04: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
No valid data found for MMP 2021-02-04
Error downloading data for MMP on 2021-02-24: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
No valid data found for MMP 2021-02-24
Error downloading data for MMP on 2021-03-06: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
No valid data found for MMP 2021-03-06
Error downloading data for MMP on 2021-03-16: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
No valid data found for MMP 2021-03-16
Error downloading data for MMP on 2021-03-26: unsupported operand type(s) for -: 'numpy.

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MMP']: Exception('%ticker%: No timezone found, symbol may be delisted')
Processing rows: 100%|██████████| 6741/6741 [1:32:22<00:00,  1.22it/s]


Error downloading data for MMP on 2021-06-14: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'
No valid data found for MMP 2021-06-14
Price updates complete.


In [ ]:
file_path = '/content/drive/MyDrive/Politician Trades/all_trades_add_days.csv'

# Save to CSV
df.to_csv(file_path, index=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35041 entries, 0 to 35040
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Politician Name            35041 non-null  object        
 1   Party                      35041 non-null  object        
 2   Chamber                    35041 non-null  object        
 3   State                      35041 non-null  object        
 4   Issuer Name                35041 non-null  object        
 5   Ticker                     35041 non-null  object        
 6   Publication Date           35041 non-null  object        
 7   Transaction Date           35041 non-null  datetime64[ns]
 8   Reporting Gap              35041 non-null  int64         
 9   Owner                      35041 non-null  object        
 10  Transaction Type           35041 non-null  object        
 11  Value Range                35041 non-null  object        
 12  Pric